In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import coo_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('../data/ai case study.csv', names=['User_Id', 'Product_Id', 'Rating', 'Timestamp'])
data.head()

,User_Id,Product_Id,Rating,Timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [3]:
def normalize(x):
    x = x.astype(float)
    x_sum = x.sum()
    x_num = x.astype(bool).sum()
    x_mean = x_sum / x_num

    if x_num == 1 or x.std() == 0:
        return 0.0
    return (x - x_mean) / (x.max() - x.min())

In [4]:
data['Rating'] = data['Rating'].astype(float)
data['Average_Rating'] = data.groupby('User_Id')['Rating'].transform(lambda x: normalize(x))
data['User_Id'] = data['User_Id'].astype('category')
data['Product_Id'] = data['Product_Id'].astype('category')

In [6]:
coo = coo_matrix((data['Average_Rating'].astype(float), (data['Product_Id'].cat.codes.copy(), data['User_Id'].cat.codes.copy())))

In [7]:
overlap_matrix = coo.astype(bool).astype(int).dot(coo.transpose().astype(bool).astype(int))

In [8]:
min_sim = 0.2
min_overlap=15
cor = cosine_similarity(coo, dense_output=False)
cor = cor.multiply(cor > min_sim)
cor = cor.multiply(overlap_matrix > min_overlap)

NameError: name 'cosine_similarity' is not defined

In [ ]:
products = dict(enumerate(data['Product_Id'].cat.categories))
products = {v: k for k, v in products.items()}

In [ ]:
np.random.seed(12)
user_id = data.User_Id.sample(1).values[0]
user_id

In [ ]:
active_user_items = data[data.User_Id == user_id].sort_values(by=['Rating', 'Timestamp'], ascending=[False, False])[:100].to_dict('records')
active_user_items

In [ ]:
product_ids = {product['Product_Id']: product['Rating'] for product in active_user_items}
product_ids